<a href="https://colab.research.google.com/github/AmadizSabino/xAI-for-Satellite-Networks/blob/main/Anomaly_Detection_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =======================================================
# Step 1 - Environment setup (Colab)
# =======================================================
!pip install --quiet --upgrade pip
!pip install --quiet streamlit plotly pandas numpy pyngrok
# Install libraries
!pip install --upgrade pip
!pip install streamlit plotly pandas pyngrok


import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

BASE_DIR = "/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses"
DASHBOARD_DIR = os.path.join(BASE_DIR, "dashboard")
os.makedirs(DASHBOARD_DIR, exist_ok=True)
DASHBOARD_DIR, os.listdir(BASE_DIR)


Mounted at /content/drive


('/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/dashboard',
 ['Anomaly_Detection_Dashboard.ipynb',
  'CIEL2_ManeuverEvents_R1.csv',
  'ThrusterTemperaturePIDs.csv',
  'AttitudePIDs.csv',
  'LabelManeuverEvents.m',
  'Archive',
  'phase3_ESA-Mission1',
  'SESGroundData_Oct-Nov_AS1.xlsx',
  'artifacts_signal_loss',
  'artifacts_sla',
  'artifacts_handover',
  'artifacts_powerfault',
  'space_weather_gfz_kp_ap_2023_v1.csv',
  'gfz_kp_ap_2023.csv',
  'ses_comm_features.csv',
  'ses_clean_10s.parquet',
  'ses_comm_anomalies.csv',
  'artifacts_jamming',
  ' 3_3_SES_Prototype_BeamHandover.ipynb',
  'ses_maneuver_features.csv',
  'gfz_kp_ap_2012-01-01_2020-02-27.csv',
  'ses_spaceweather_dataset.csv',
  'artifacts_spaceweather',
  'dashboard',
  '3_3_SES_Prototype_SignalLoss.ipynb',
  '3_3_SES_Jamming_Interference.ipynb',
  '3_3_SES_Space_Weather.ipynb',
  '3_3_SES_Prototype_SLA_Proxy.ipynb',
  'artifacts_capacity',
  '3_3_SES_Risk_Aware_Capacity_Advisor.ipynb',
  'artifacts_stre

In [9]:
!ls "$DASHBOARD_DIR"

!ls "$BASE_DIR"

app.py	OLD_1_app.py  OLD_3app.py  OLD_5app.py	OLD_7app.py
assets	OLD_2app.py   OLD_4app.py  OLD_6app.py
 3_3_SES_Jamming_Interference.ipynb
' 3_3_SES_Prototype_BeamHandover.ipynb'
 3_3_SES_Prototype_SignalLoss.ipynb
 3_3_SES_Prototype_SLA_Proxy.ipynb
 3_3_SES_Prototype_StressIndex.ipynb
 3_3_SES_Risk_Aware_Capacity_Advisor.ipynb
 3_3_SES_Space_Weather.ipynb
 Anomaly_Detection_Dashboard.ipynb
 Archive
 artifacts_capacity
 artifacts_handover
 artifacts_jamming
 artifacts_powerfault
 artifacts_signal_loss
 artifacts_sla
 artifacts_spaceweather
 artifacts_stress
 AttitudePIDs.csv
 CIEL2_ManeuverEvents_R1.csv
 dashboard
 gfz_kp_ap_2012-01-01_2020-02-27.csv
 gfz_kp_ap_2023.csv
 LabelManeuverEvents.m
 OLD_Anomaly_Detection_Dashboard.ipynb
 phase3_ESA-Mission1
 ses_clean_10s.parquet
 ses_comm_anomalies.csv
 ses_comm_features.csv
 SESGroundData_Oct-Nov_AS1.xlsx
 ses_maneuver_features.csv
 ses_spaceweather_dataset.csv
 space_weather_gfz_kp_ap_2023_v1.csv
 ThrusterTemperaturePIDs.csv


# Nova secção

In [37]:
# =======================================================
# Step 2 - create app.py file
# =======================================================

import textwrap
import os

# Define BASE_DIR and DASHBOARD_DIR within this cell for independence
BASE_DIR = "/content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses"
DASHBOARD_DIR = os.path.join(BASE_DIR, "dashboard")

app_code_raw = '''
import streamlit as st
import pandas as pd
import numpy as np
from pathlib import Path
import plotly.express as px

# ------------------------------
# Paths and helpers
# ------------------------------

BASE_DIR = Path(__file__).resolve().parent.parent

def load_csv(relative_paths, parse_dates=None):
    paths = relative_paths if isinstance(relative_paths, (list, tuple)) else [relative_paths]
    for rel in paths:
        p = BASE_DIR / rel
        if p.exists():
            try:
                return pd.read_csv(p, parse_dates=parse_dates)
            except Exception:
                return None
    return None

def load_image_path(relative_paths):
    paths = relative_paths if isinstance(relative_paths, (list, tuple)) else [relative_paths]
    for rel in paths:
        p = BASE_DIR / rel
        if p.exists():
            return str(p)
    return None

# ------------------------------
# Page wide layout
# ------------------------------

st.set_page_config(page_title="Satellite Anomaly Dashboard",
                   layout="wide")

# Sidebar navigation
st.sidebar.title("Anomaly Dashboard")
view = st.sidebar.radio(
    "Select a view",
    [
        "Overview",
        "Signal Loss",
        "Jamming / Interference",
        "SLA Breach",
        "Beam Handover",
        "Space Weather",
        "Risk-aware Capacity Advisor",
        "Stress Index & Joint Risk Radar",
    ],
)

# Demo filters (top of sidebar)
with st.sidebar.expander("Demo filters", expanded=True):
    st.caption("In a live system these would filter the underlying data.")
satellite = st.sidebar.selectbox("Satellite (aggregated data)", ["M003", "M008", "M015", "M017", "ALL"])
time_window = st.sidebar.selectbox("Time window", ["Last 24h", "Last 7 days", "Full dataset"])

# ------------------------------
# Alert card helper
# ------------------------------

def render_alerts(df, id_col, time_col, severity_col, title, usecase_key, max_rows=3):
    """
    Render a compact list of alert cards from a dataframe with id, time and severity columns.
    """
    if df is None or df.empty:
        # Small demo fallback set if nothing is available.
        demo = pd.DataFrame(
            {
                id_col: [f"ALERT-{i+1}" for i in range(3)],
                time_col: pd.date_range("2021-11-01", periods=3, freq="H"),
                severity_col: ["high", "medium", "medium"],
            }
        )
        df_to_show = demo
    else:
        df_to_show = df.sort_values(time_col, ascending=False).head(max_rows)

    st.caption(title)

    # IMPORTANT: this loop defines `row`, so everything below can use it
    for i, (_, row) in enumerate(df_to_show.iterrows()):
        with st.container():
            markdown_content = f"""**{row[id_col]}**
Time: {row[time_col]}
Severity: {row[severity_col]}"""
            st.markdown(markdown_content)

            col_a, col_b = st.columns([1, 2])
            with col_a:
                st.button("Acknowledge", key=f"{usecase_key}_ack_{i}")
            with col_b:
                st.caption(
                    "Typical action: open NOC ticket, attach evidence, notify on-call engineer."
                )


# ------------------------------
# Overview page
# ------------------------------

def page_overview():
    col_left, col_right = st.columns([2, 1])

    with col_left:
        st.title("Explainable AI for Satellite Networks Anomaly Detection")
        st.markdown(
          """
            Monitoring based on ground telemetry, space weather and SLA metrics.
            The goal is to give the Network Operations Center early insight into
            issues that affect availability and customer experience.
          """
        )

        st.divider()

        st.subheader("What this prototype does")

        st.markdown(
            """
- Tracks a set of anomaly use cases: signal loss, jamming or interference, SLA breach, beam handover issues,
  space weather risk for maneuvers, capacity pressure and a combined stress index.
- Uses historical SES data to learn what healthy behaviour looks like, then scores new windows for risk.
- Uses explainable AI (mainly SHAP heatmaps) so operators can see **why** a window is flagged.
            """
        )

        st.divider()
        st.subheader("High level anomaly detection performance (test windows)")

        st.markdown(
            """
**How to read these metrics**

- **PR-AUC** (Precision–Recall area): how well the model lifts true anomalies above noise.
  This is the main metric for rare events such as signal loss or SLA breach.

- **ROC-AUC**: overall ability to separate normal vs. anomalous behaviour across all thresholds.
  This can look optimistic when anomalies are very rare.

- **Event precision / recall**:
  Precision answers *“of the alerts raised, how many were real?”*;
  Recall answers *“of all real events, how many did we catch?”*.

In this prototype the models are tuned towards **high precision** so that operators can trust an alert,
even if that means some events are missed (moderate recall) to avoid flooding the NOC with false positives.
            """
        )

        metrics = [
            ("Signal Loss model",        0.72, 0.83, 0.93, 0.13),
            ("SLA early warning",       0.25, 0.86, 1.00, 0.03),
            ("Beam Handover anomalies", 0.022, 0.71, 0.50, 0.012),
        ]

        interpretations = {
            "Signal Loss model": (
                "Strong at prioritising true signal-loss events over noise. "
                "Most alerts are real (high precision), but it currently catches only a subset of all events "
                "(moderate recall). Designed as a conservative early-warning signal."
            ),
            "SLA early warning": (
                "Identifies some windows at risk of SLA degradation. Alerts are very rare but almost always correct "
                "(precision close to 1.0), so it behaves as a highly conservative early-warning indicator rather than "
                "a complete SLA monitor."
            ),
            "Beam Handover anomalies": (
                "Weaker performance, which is expected given the extreme rarity and complexity of handover issues. "
                "Surfaces some interesting cases but still misses many true problems and generates some false alerts. "
                "Marked as *prototype / for further tuning*."
            ),
        }

        m_cols = st.columns(len(metrics))
        for col, (name, pr, roc, p, r) in zip(m_cols, metrics):
            with col:
                st.markdown(f"**{name}**")
                st.metric("PR-AUC", f"{pr:.3f}")
                st.metric("ROC-AUC", f"{roc:.3f}")
                st.caption(f"Event precision / recall: **{p:.2f} / {r:.2f}**")
                with st.expander("Plain-English summary", expanded=False):
                    st.write(interpretations[name])

        st.divider()
        st.subheader("Operational notes (prototype)")

        op_col1, op_col2 = st.columns([2, 1])
        with op_col1:
            st.markdown(
              """
                In a production live environment this dashboard would be connected to the NOC data lake
                and refreshed continuously. Operator feedback on alerts would be stored for model retraining
                and for tracking false positives.
              """
            )
            _ = st.text_area(
                "Operator feedback (demo only, not persisted)",
                placeholder="Example: Signal loss alert at 05:50 UTC was a false alarm – modem under maintenance.",
            )
        with op_col2:
            st.markdown("**Data freshness**")
            st.markdown("- Ground telemetry window: **Oct–Nov 2021**")
            st.markdown("- Space weather window: **2012–2020**")
            st.markdown("- Last model run (prototype): **2025-11-25 UTC**")

    with col_right:
        gif_url = "https://i.gifer.com/AHJv.gif"
        st.image(gif_url, caption="Orbital view (demo)", use_container_width=True)

# ------------------------------
# Signal Loss page
# ------------------------------

def page_signal_loss():
    st.title("Signal Loss – modem level detection")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        The model watches a small set of modem power and quality indicators over time.
        When they drift away from their usual pattern, the window is flagged as a potential signal loss scenario.
        It focuses on bursts and ramps in input and output power rather than single out-of-range samples.
      """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
      """
        - Impacts service availability and can trigger SLA penalties.
        - Distinguishing true RF issues from planned maintenance is difficult at scale.
        - Manual triage across many modems and beams is time-consuming; automation helps focus operator attention.
      """
    )

    st.markdown("### How this prototype works")
    st.markdown(
      """
        - Features are built from modem IN and OUT power statistics over short windows (mean, min, max, slopes).
        - An autoencoder-style model learns the typical pattern and assigns an anomaly score to each window.
        - A threshold and eventisation logic convert noisy scores into a small number of operational alerts.
      """
    )

    col_main, col_side = st.columns([2, 1])

    with col_main:
        st.markdown("### SHAP heatmaps – drivers of signal loss")

        event_img = load_image_path("artifacts_signal_loss/signal_loss_event_heatmap.png")
        cont_img = load_image_path("artifacts_signal_loss/signal_loss_continuous_heatmap.png")

        if event_img:
            st.image(event_img, caption="Signal Loss – SHAP heatmap around one event", use_container_width=True)
        if cont_img:
            st.image(cont_img, caption="Signal Loss – continuous SHAP importance over time", use_container_width=True)

        st.markdown(
            """
The heatmaps show **which modem statistics pushed the model towards signal loss**.

- **Rows**: modem-level features (IN/OUT power statistics, possibly per-channel aggregates).
- **Columns**: time steps within the analysed window.
- **Warm colours (red/orange)**: features that *increase* the anomaly score in that window.
- **Cool colours (blue)**: features that *reduce* it or confirm normal behaviour.

Operators can quickly see whether a suspected event is driven by a single modem, many modems together, or noisy data.
            """
        )

        st.markdown("### Example trend")

        scores = load_csv("artifacts_signal_loss/test_scores_raw.csv", parse_dates=["timestamp"])

        if scores is not None:
            if "timestamp" in scores.columns:
                scores = scores.rename(columns={"timestamp": "time"})
            if "proba_raw" in scores.columns:
                scores = scores.rename(columns={"proba_raw": "anomaly_score"})

            if "time" in scores.columns and "anomaly_score" in scores.columns:
                scores = scores.sort_values("time").tail(1000)
                fig = px.line(
                    scores,
                    x="time",
                    y="anomaly_score",
                    title="Recent signal-loss anomaly scores",
                )
                fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
                st.plotly_chart(fig, use_container_width=True)
            else:
                st.caption("Signal-loss scores CSV found, but expected columns are missing; trend chart skipped.")
        else:
            st.caption("Signal-loss scores CSV not found; trend chart skipped in this environment.")

    with col_side:
        gif_url = "https://i.gifer.com/K6mM.gif"
        st.image(gif_url, caption="Signal Loss illustration (demo)", use_container_width=True)

        st.markdown("### Alerts and suggested actions")

        events = load_csv(
            ["artifacts_signal_loss/test_eventized_scores.csv", "artifacts_signal_loss/test_scores.csv"],
            parse_dates=["t_start", "t_end"],
        )
        if events is not None:
            events["time_center"] = events["t_start"]
            events["severity"] = np.where(events.get("label", 1) == 1, "high", "medium")
            events["id"] = events.get("modem", "Unknown modem")
            alerts_df = events[["id", "time_center", "severity"]]
        else:
            alerts_df = None

        render_alerts(alerts_df, "id", "time_center", "severity", "Recent high-risk windows", "signal_loss")

        st.markdown(
          """
            - Correlate with weather, maintenance and pointing information.
            - If multiple modems on the same beam are affected, escalate as potential RF impairment.
            - If a single modem only, open a customer ticket and investigate the terminal side first.
          """
        )

# ------------------------------
# Jamming / interference page
# ------------------------------

def page_jamming():
    st.title("Jamming / Interference – spectrum-level anomalies")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        The jamming detector looks for unusual energy patterns in a subset of modem outputs.
        Instead of analysing full spectra, it uses modem statistics as a proxy for interference on the uplink or downlink.
      """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
      """
        - Intentional or accidental jamming can degrade whole beams and affect many customers at once.
        - Early detection enables geolocation, mitigation and reconfiguration before the impact widens.
        - Weak or intermittent interferers can remain unnoticed for hours without automated monitoring.
      """
    )

    st.markdown("### How this prototype works")
    st.markdown(
      """
        - An unsupervised model learns the joint behaviour of modem statistics during quiet periods.
        - Windows where many channels move together in an unusual way receive a high anomaly score.
        - SHAP explanations highlight which channels and time slices drove each alarm.
      """
    )

    col_main, col_side = st.columns([2, 1])

    with col_main:
        st.markdown("### SHAP heatmaps – interference patterns")

        event_img = load_image_path("artifacts_jamming/jamming_event_heatmap.png")
        cont_img = load_image_path("artifacts_jamming/jamming_continuous_heatmap.png")

        if event_img:
            st.image(
                event_img,
                caption="Jamming – SHAP heatmap around a suspected interference event",
                use_container_width=True,
            )
        if cont_img:
            st.image(
                cont_img,
                caption="Jamming – continuous SHAP importance over time",
                use_container_width=True,
            )

        st.markdown(
            """
These SHAP heatmaps show **which modem statistics contributed most to the model's decision** that a time
window is suspicious for interference.

**Axes and colours**

- **Rows**: modem- or channel-level statistics (e.g. power, quality, error counters) aggregated over short windows.
- **Columns**: time steps within the analysed window.
- **Warm colours (red/orange)**: features that *increase* the jamming anomaly score.
- **Cool colours (blue)**: features that *reduce* the score or confirm normal behaviour.

**Typical jamming signatures**

- **Wide horizontal bands across several rows at the same time**
  → Many channels show abnormal behaviour together → suggestive of **wideband interference** or a jammer affecting
  a whole beam or polarisation.

- **Strong activity concentrated in a small number of rows**
  → Only a few channels or carriers are affected → more indicative of **localised carrier problems** or
  customer-specific issues.

- **Short, intense vertical stripes**
  → Sudden bursts of abnormal energy → could indicate **short-lived or bursty interferers**.

**Operational use**

- Confirm whether an alert is driven by a **single noisy modem** or a **coherent pattern across many channels**.
- Prioritise events with **multi-row, multi-time anomalies** as stronger candidates for real jamming.
- Use the SHAP rows to decide which beams/channels to inspect first in external spectrum monitoring tools.
            """
        )

    with col_side:
        st.markdown("### Alerts and suggested actions")

        events = load_csv("artifacts_jamming/test_eventized_scores.csv", parse_dates=["t_start", "t_end"])
        if events is not None:
            events["time_center"] = events["t_start"]
            events["severity"] = np.where(events.get("label", 1) == 1, "high", "medium")
            events["id"] = events.get("beam", "Unknown beam")
            alerts_df = events[["id", "time_center", "severity"]]
        else:
            alerts_df = None

        render_alerts(alerts_df, "id", "time_center", "severity", "Recent high-risk windows", "jamming")

        st.markdown(
          """
            - Cross-check with spectrum monitoring tools and confirm on a waterfall view.
            - Start geolocation if multiple beams show correlated interference.
            - Coordinate with customers to move critical carriers if necessary.
          """
        )

# ------------------------------
# SLA Breach page
# ------------------------------

def page_sla():
    st.title("SLA Breach – throughput early warning")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        This model watches a throughput proxy KPI and compares it against SLA thresholds learned from historical data.
        Instead of only reacting once a customer complains, it aims to provide a few minutes of warning that traffic
        is likely to fall below an agreed level.
      """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
      """
        - SLA violations impact revenue and customer satisfaction.
        - Without early warning, the NOC is often reactive rather than proactive.
        - Even a small lead time is valuable to reroute traffic or add capacity.
      """
    )

    col_main, col_side = st.columns([2, 1])

    with col_main:
        st.markdown("### SLA thresholds and breaches (example windows)")

        sla_df = load_csv("artifacts_sla/sla_breach_events.csv", parse_dates=["start", "end"])
        if sla_df is not None:
            fig = px.scatter(
                sla_df.sort_values("start").head(200),
                x="start",
                y="duration_s",
                color="severity" if "severity" in sla_df.columns else None,
                title="Example windows leading into SLA breaches",
            )
            fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("No SLA breach CSV found in artifacts_sla/. Showing explanations only.")

        st.markdown("### SHAP heatmaps – drivers of SLA risk")

        event_img = load_image_path("artifacts_sla/sla_event_heatmap.png")
        cont_img = load_image_path("artifacts_sla/sla_continuous_heatmap.png")

        if event_img:
            st.image(
                event_img,
                caption="SLA – SHAP heatmap around one breach",
                use_container_width=True,
            )
        if cont_img:
            st.image(
                cont_img,
                caption="SLA – continuous SHAP importance over time",
                use_container_width=True,
            )

        st.markdown(
            """
The SLA SHAP heatmaps explain **which KPIs pushed the model towards predicting an SLA breach** in each time window.

**Axes and colours**

- **Rows**: engineered features such as current throughput level, recent minimum throughput, volatility, slope and
  short-term history of the KPI.
- **Columns**: time steps leading up to and after a potential SLA issue.
- **Warm colours (red/orange)**: values that *increase* the probability of an SLA breach.
- **Cool colours (blue)**: values that *decrease* the probability and support a “healthy” interpretation.

**Typical SLA risk signatures**

- **Deep blue patches on throughput rows just before a breach**
  → Throughput is consistently below the model’s expectation for a healthy window → strong breach evidence.

- **Red spikes on slope or volatility features**
  → Sharp drops or unstable throughput over a short period → treated as early-warning symptoms.

- **Sustained warm colours across several throughput-related rows**
  → Combination of low level, high volatility and negative trend → degraded and unstable service.

**Operational use**

- Separate **hard SLA breaches** from **borderline unstable windows** that may soon degrade.
- Understand whether a predicted risk is driven mainly by **absolute level** or by **instability and sharp drops**.
- Justify proactive actions (capacity boost, rerouting, customer communication) before a formal breach occurs.
            """
        )

    with col_side:
        st.markdown("### Alerts and suggested actions")

        sla_df = load_csv("artifacts_sla/sla_breach_events.csv", parse_dates=["start", "end"])
        if sla_df is not None:
            sla_df = sla_df.copy()
            sla_df["time_center"] = sla_df["start"]
            sla_df["severity"] = np.where(sla_df.get("breach_flag", 1) == 1, "high", "medium")
            sla_df["id"] = sla_df.get("kpi_id", "SLA throughput")
            alerts_df = sla_df[["id", "time_center", "severity"]]
        else:
            alerts_df = None

        render_alerts(alerts_df, "id", "time_center", "severity", "Current SLA risk windows", "sla")

        st.markdown(
          """
            - If predicted breach is local to one beam, check utilisation and consider temporary capacity boost.
            - If several beams show risk, escalate to network planning and investigate ground segment issues.
          """
        )

# ------------------------------
# Beam Handover page
# ------------------------------

def page_handover():
    st.title("Beam Handover – quality after beam changes")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        Whenever the satellite or network controller moves a terminal from one beam to another,
        there is a short period where throughput can dip. This model tracks handovers and highlights those
        where throughput drops more than expected or recovers slowly.
      """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
      """
        - Poorly behaving handovers can create repeated short outages that are hard to diagnose.
        - They often affect mobile or aeronautical customers, which are especially sensitive to quality.
        - Early visibility enables targeted tuning of handover parameters.
      """
    )

    col_main, col_side = st.columns([2, 1])

    with col_main:
        st.markdown("### SHAP heatmaps – quality around handovers")

        event_img = load_image_path("artifacts_handover/handover_event_heatmap.png")
        cont_img = load_image_path("artifacts_handover/handover_continuous_heatmap.png")

        if event_img:
            st.image(
                event_img,
                caption="Beam Handover – SHAP heatmap around one anomalous handover",
                use_container_width=True,
            )
        if cont_img:
            st.image(
                cont_img,
                caption="Beam Handover – continuous SHAP importance over time",
                use_container_width=True,
            )

        st.markdown(
            """
These heatmaps show how the model evaluates handover quality **before, during and after** the beam change.

**Axes and structure**

- **Rows**: features such as throughput before handover, throughput after handover, percentage drop, recovery time,
  packet loss, and related quality indicators.
- **Columns**: time windows ordered around the handover moment. A vertical reference separates **pre-handover**
  and **post-handover** windows.
- **Warm colours (red/orange)**: features that *increase* the anomaly score (concern about handover quality).
- **Cool colours (blue)**: features that push the model towards normal behaviour.

**Typical handover anomaly signatures**

- **Warm regions immediately after the handover time**
  → Post-handover windows where throughput is low or recovery is slow → the new beam is not delivering expected quality.

- **Strong warm colours on “drop percentage” or “minimum throughput after handover” rows**
  → Larger-than-expected drop from pre- to post-handover throughput → potential mis-tuned parameters, coverage edge
  effects, or interference.

- **Sustained warm colours across multiple post-handover windows**
  → Prolonged poor performance rather than a brief transient → operationally critical and worth engineering follow-up.

**Operational use**

- Distinguish events where conditions were poor **before** the handover from those where the handover itself degraded quality.
- Prioritise investigation on handovers where **recovery time and drop percentage stay red** over several windows.
- Provide feedback on specific events where explanations do not match operator perception, feeding future retraining.
            """
        )

    with col_side:
        st.markdown("### Alerts and suggested actions")

        events = load_csv("artifacts_handover/handover_table.csv", parse_dates=["t"])
        if events is not None:
            events = events.tail(200)
            events["time_center"] = events["t"]
            events["severity"] = np.where(events["drop_pct"].abs() > 0.05, "high", "medium")
            events["id"] = events.get("beam_id", "Unknown beam")
            alerts_df = events[["id", "time_center", "severity"]]
        else:
            alerts_df = None

        render_alerts(alerts_df, "id", "time_center", "severity", "Recent anomalous handovers", "handover")

        st.markdown(
          """
            - Check whether the same customer or route is affected repeatedly.
            - Inspect handover timing relative to satellite motion and beam footprints.
            - Consider adjusting hysteresis or thresholds for problem beams.
          """
        )

# ------------------------------
# Space Weather page
# ------------------------------

def page_space_weather():
    st.title("Space Weather – maneuver risk")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        Space weather indices such as Kp and ap capture geomagnetic activity.
        This prototype links those indices with thruster temperature and attitude error during station-keeping and
        unload maneuvers, to flag windows where maneuver risk may be elevated.
      """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
      """
        - During strong geomagnetic storms the environment around the spacecraft changes and control margins shrink.
        - Maneuvers executed in those periods may have higher fuel usage or tighter thermal constraints.
        - A simple risk indicator helps flight dynamics teams choose safer windows.
      """
    )

    col_main, col_side = st.columns([2, 1])

    with col_main:
        st.markdown("### SHAP heatmaps – maneuver risk under space weather")

        event_img = load_image_path("artifacts_spaceweather/spaceweather_risky_heatmap.png")
        cont_img = load_image_path("artifacts_spaceweather/spaceweather_continuous_heatmap.png")

        if event_img:
            st.image(
                event_img,
                caption="Space Weather – SHAP heatmap for top risky maneuvers",
                use_container_width=True,
            )
        if cont_img:
            st.image(
                cont_img,
                caption="Space Weather – continuous SHAP importance over time",
                use_container_width=True,
            )

        st.markdown(
            """
These heatmaps explain **why the model considers certain maneuvers to be higher risk** from a space-weather
perspective.

**Axes and feature types**

- **Rows** typically include:
  - Thruster temperature statistics during the maneuver
  - Attitude error metrics
  - Space weather indices (Kp, ap) and storm flags
  - Short-term history of geomagnetic activity before the maneuver
- **Columns**: maneuvers or time windows considered by the model.
- **Warm colours (red/orange)**: values that push the classifier towards the “risky” class.
- **Cool colours (blue)**: values that support a nominal or low-risk classification.

**Typical risky maneuver signatures**

- **Warm blocks on thruster temperature and attitude error rows during a maneuver**
  → Thrusters running hotter or control margins tighter than usual → closer to engineering limits.

- **Sustained warm bands on Kp or storm-related features before and during maneuvers**
  → Prolonged geomagnetic disturbance around the maneuver window → external conditions make control more challenging.

- **Combined warm patterns across both spacecraft and space-weather rows**
  → Internal telemetry and external indices both pointing towards increased risk → maneuvers where postponement
  or extra monitoring may be most justified.

**Operational use**

- Distinguish maneuvers that are risky **due to internal telemetry only** from those that are risky **due to disturbed space weather**.
- Better justify decisions to **delay non-urgent maneuvers** when geomagnetic activity is high.
- Build intuition over time about which combinations of Kp, attitude error and thruster behaviour tend to precede
  operational issues.
            """
        )

    with col_side:
        st.markdown("### Alerts and suggested actions")

        maneuvers = load_csv("ses_spaceweather_dataset.csv", parse_dates=["time"])
        if maneuvers is not None and "risk_score" in maneuvers.columns:
            maneuvers = maneuvers.sort_values("time", ascending=False).head(20)
            maneuvers["time_center"] = maneuvers["time"]
            maneuvers["severity"] = np.where(maneuvers["risk_score"] > 0.6, "high", "medium")
            maneuvers["id"] = maneuvers["maneuver_type"]
            alerts_df = maneuvers[["id", "time_center", "severity"]]
        else:
            alerts_df = None

        render_alerts(alerts_df, "id", "time_center", "severity", "Upcoming or recent risky maneuvers", "spaceweather")

        st.markdown(
          """
            - Avoid planning non-urgent maneuvers during long periods with high Kp.
            - Coordinate with ground segment teams when storm intensity is high, as link margins may also be affected.
          """
        )

# ------------------------------
# Risk aware capacity advisor
# ------------------------------

def synth_capacity_demo():
    \"\"\"Create synthetic capacity and demand for illustration if no CSV exists.\"\"\"
    idx = pd.date_range("2021-10-25", "2021-11-01", freq="H")
    beams = ["Beam-A", "Beam-B", "Beam-C"]
    rows = []
    rng = np.random.default_rng(42)
    for b in beams:
        base_cap = rng.uniform(200, 260)
        for t in idx:
            demand = base_cap * rng.uniform(0.4, 1.2)
            cap = base_cap * rng.uniform(0.9, 1.1)
            rows.append({"time": t, "beam": b, "capacity": cap, "demand": demand})
    df = pd.DataFrame(rows)
    df["headroom"] = df["capacity"] - df["demand"]
    df["risk_index"] = 1.0 - (df["headroom"] / df["capacity"]).clip(0, 1)
    return df

def page_capacity():
    st.title("Risk-aware capacity advisor (prototype)")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        This page tracks how close traffic demand is to the available capacity on each beam or region.
        When demand rises towards the capacity limit, the risk of congestion and throughput degradation increases.
      """
    )

    st.markdown("### What this view shows")
    st.markdown(
        """
This view highlights beams or regions where **traffic demand is getting close to the available capacity**.

**How to read the chart:**
- **Capacity** is the estimated bandwidth the system can safely deliver.
- **Demand** is how much bandwidth customers are actually trying to use.
- **Risk index** reflects how tight the margin is between demand and capacity:
  - Low risk → Plenty of room
  - Medium risk → Demand increasing into the normal operating range
  - High risk → Demand is close to or above the safe operating limit

**Operational meaning:**
- A rising risk index signals beams that may soon become overloaded.
- Persistent high risk may require:
  - Temporary capacity boosts
  - Rerouting or reshaping traffic
  - Escalation to planning/engineering teams

This gives early visibility into congestion risks *before* customers feel an impact.
        """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
        """
When traffic demand rises close to the available capacity, **customer experience can degrade quickly**, even before
traditional alarms or customer tickets appear.

This view helps operators to:

- Identify beams or regions that are **about to become congested**
- Act before customers notice problems
- Prioritise which beams require operational attention
- Coordinate with planning or engineering teams when capacity is consistently tight
- Consider mitigations such as temporary capacity boosts or rerouting traffic

Early visibility allows the NOC to stay **proactive** rather than reactive.
        """
    )

    df = load_csv("artifacts_capacity/capacity_risk_demo.csv", parse_dates=["time"])
    if df is None:
        df = synth_capacity_demo()

    beam_options = ["ALL"] + sorted(df["beam"].unique().tolist())
    beam = st.selectbox("Beam or region", beam_options)
    horizon_label = st.selectbox("Forecast horizon", ["Next 6 hours", "Next 24 hours", "Next 72 hours"])
    horizon_hours = 6 if "6" in horizon_label else (24 if "24" in horizon_label else 72)

    latest_time = df["time"].max()
    window_start = latest_time - pd.Timedelta(hours=horizon_hours)
    df_win = df[df["time"].between(window_start, latest_time)].copy()
    if beam != "ALL":
        df_win = df_win[df_win["beam"] == beam]

    st.markdown("### Capacity, demand and risk over selected horizon")
    fig = px.line(
        df_win,
        x="time",
        y=["capacity", "demand", "risk_index"],
        labels={"value": "Mbps / risk", "variable": "Series"},
    )
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0), legend=dict(orientation="h", yanchor="bottom", y=1.02))
    st.plotly_chart(fig, use_container_width=True)

    st.markdown(
      """
        Risk index values close to 1.0 indicate little spare headroom between demand and capacity.
        In a production system this would be driven by a proper demand forecast and spectrum plan optimisation model.
      """
    )

    st.markdown("### Alerts and suggested actions")
    df_last = df_win.sort_values("time", ascending=False).head(10)
    df_last["severity"] = np.where(df_last["risk_index"] > 0.8, "high", "medium")
    df_last["id"] = df_last["beam"]
    df_last["time_center"] = df_last["time"]
    alerts_df = df_last[["id", "time_center", "severity"]]
    render_alerts(alerts_df, "id", "time_center", "severity", "Beams with highest short-term capacity risk", "capacity")

    st.markdown("#### Animated illustration placeholder")
    st.caption("Placeholder for an animation showing beams changing colour as capacity pressure increases.")

# ------------------------------
# Stress index page
# ------------------------------

def synth_stress_demo():
    \"\"\"Create synthetic stress index for illustration if no CSV exists.\"\"\"
    idx = pd.date_range("2021-10-25", "2021-11-01", freq="H")
    rng = np.random.default_rng(123)
    df = pd.DataFrame({"time": idx})
    df["signal_loss_risk"] = rng.uniform(0, 0.6, len(idx))
    df["jamming_risk"] = rng.uniform(0, 0.5, len(idx))
    df["sla_risk"] = rng.uniform(0, 0.7, len(idx))
    df["capacity_risk"] = rng.uniform(0, 0.8, len(idx))
    df["stress_index"] = df[["signal_loss_risk", "jamming_risk", "sla_risk", "capacity_risk"]].max(axis=1)
    return df

def page_stress():
    st.title("Stress index and joint risk radar")

    st.markdown("### What this use case monitors")
    st.markdown(
      """
        The stress index combines signals from several models into one compact view.
        Instead of four or five separate alert streams, the NOC gets an at-a-glance indicator of how stressed the
        network is over time on each satellite.
      """
    )

    st.markdown("### What this view shows")
    st.markdown(
        """
The stress index gives a **single combined view** of several anomaly indicators:
signal loss, jamming/interference, SLA degradation risk, and capacity pressure.

**How to read the chart:**
- The line moves up whenever **any** underlying model shows elevated risk.
- Peaks indicate times when the network experienced:
  - Unusual modem behaviour
  - Interference events
  - SLA-related throughput issues
  - Capacity tightening

This provides a fast, at-a-glance understanding of overall satellite network health.
        """
    )

    st.markdown("### Why this matters for operators")
    st.markdown(
        """
Operators normally need to monitor several different anomaly indicators separately
(signal loss, interference, SLA risk, capacity pressure). The **stress index combines these signals**,
helping operators instantly understand whether the network is behaving normally or under pressure.

This view helps operators to:

- Gauge **overall network health** at a glance
- Detect **multi-beam or network-wide issues** quickly
- Decide which use-case page requires immediate attention
- Spot patterns that might be missed when looking at individual models
- Prioritise responses during periods of operational stress

A rising stress index is a clear signal that the NOC should investigate further.
        """
    )

    df = load_csv("artifacts_stress/stress_index_demo.csv", parse_dates=["time"])
    if df is None:
        df = synth_stress_demo()

    st.markdown("### Stress index over time")
    fig = px.line(df, x="time", y="stress_index", title="Combined stress index")
    fig.update_layout(margin=dict(l=0, r=0, t=40, b=0))
    st.plotly_chart(fig, use_container_width=True)

    st.markdown(
      """
        Peaks in the stress index correspond to periods where at least one underlying model showed elevated risk.
        In a production deployment this page would link back to the individual use case pages for root cause analysis.
      """
    )

    st.markdown("### Alerts and suggested actions")
    df_last = df.sort_values("time", ascending=False).head(10)
    df_last["severity"] = np.where(df_last["stress_index"] > 0.8, "high", "medium")
    df_last["id"] = "SAT net"
    df_last["time_center"] = df_last["time"]
    alerts_df = df_last[["id", "time_center", "severity"]]
    render_alerts(alerts_df, "id", "time_center", "severity", "Most stressed recent periods", "stress")

    st.markdown("#### Animated illustration placeholder")
    st.caption("Placeholder for a radar-style animation that brightens when the stress index rises.")

# ------------------------------
# Router
# ------------------------------

if view == "Overview":
    page_overview()
elif view == "Signal Loss":
    page_signal_loss()
elif view == "Jamming / Interference":
    page_jamming()
elif view == "SLA Breach":
    page_sla()
elif view == "Beam Handover":
    page_handover()
elif view == "Space Weather":
    page_space_weather()
elif view == "Risk-aware Capacity Advisor":
    page_capacity()
elif view == "Stress Index & Joint Risk Radar":
    page_stress()
'''

# Remove common leading whitespace from app_code_raw
app_code = textwrap.dedent(app_code_raw)

with open(f"{DASHBOARD_DIR}/app.py", "w") as f:
    f.write(app_code)

print("Saved:", f"{DASHBOARD_DIR}/app.py")


Saved: /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/dashboard/app.py


In [38]:
# =======================================================
# Step 3 - Install streamlit
# =======================================================

!pip install streamlit -q
!python -c "import streamlit; print('Streamlit version:', streamlit.__version__)"
!python -m streamlit run /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/dashboard/app.py \
    --server.port 8501 --server.address 0.0.0.0 &>/tmp/streamlit.log &
!tail -n 50 /tmp/streamlit.log

Streamlit version: 1.51.0


In [39]:
# =======================================================
# Step 4 - Install cloudfare
# =======================================================
# Download cloudflared binary
!curl -L https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -o cloudflared
!chmod +x cloudflared


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 39.3M  100 39.3M    0     0  93.7M      0 --:--:-- --:--:-- --:--:-- 93.7M


In [40]:
# =======================================================
# Step 5 - Run streamlit
# =======================================================

!python -m streamlit run /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/dashboard/app.py \
    --server.port 8501 --server.address 0.0.0.0 &>/tmp/streamlit.log &

!tail -n 50 /tmp/streamlit.log


In [41]:
!tail -n 50 /tmp/streamlit.log

!ps aux | grep streamlit



2025-12-03 10:53:48.294 Port 8501 is already in use
root       18708  6.2  0.5 232440 70220 ?        S    10:53   0:02 python3 -m streamlit run /content/drive/MyDrive/Colab_Notebooks/Thesis-AI/phase3_ses/dashboard/app.py --server.port 8501 --server.address 0.0.0.0
root       18949  0.0  0.0   7376  3592 ?        S    10:53   0:00 /bin/bash -c ps aux | grep streamlit
root       18951  0.0  0.0   6484  2528 ?        S    10:53   0:00 grep streamlit


In [ ]:
# =======================================================
# Step 6 - Launch cloudfare
# =======================================================

!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate


2025-12-03T10:53:58Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-12-03T10:53:58Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-12-03T10:54:03Z INF +--------------------------------------------------------------------------------------------+
2025-12-03T10:54:03Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-12-03T10:54:03Z INF |  https://qty-northwest-psychiatry-pas.trycloudflare.co